In [50]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import layers, Model
import keras.backend as K
import numpy as np

In [51]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [52]:
IMG_SIZE = 32
GRAYSCALE = 255
image_input_size = (IMG_SIZE, IMG_SIZE, 1)

# images are in 28x28 ints between 0-255 (grayscale)
# we want them scaled between 0-1 and padded to 32x32 (28+4)

def preprocess_images(images):
    return np.pad(images.astype("float32") / GRAYSCALE, ((0, 0), (2, 2), (2, 2)), constant_values=0.0)

In [53]:
x_train = preprocess_images(x_train)
x_test = preprocess_images(x_test)

In [54]:
x_train[0].shape

(32, 32)

In [55]:
encoder_input = layers.Input(shape=image_input_size, name="encoder_input")

In [56]:
x = layers.Conv2D(IMG_SIZE, (3, 3), strides=2, activation='relu', padding='same')(encoder_input)
x = layers.Conv2D(IMG_SIZE * 2, (3, 3), strides=2, activation='relu', padding='same')(x)
x = layers.Conv2D(IMG_SIZE * 4, (3, 3), strides=2, activation='relu', padding='same')(x)
shape_before_flattening = K.int_shape(x)[1:]
shape_before_flattening

(4, 4, 128)

In [57]:
x = layers.Flatten()(x)

In [58]:
encoder_output = layers.Dense(2, name="encoder_output")(x)

In [59]:
encoder = Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
Total params: 96,770
Trainable params: 96,770
Non-trainable

In [60]:
decoder_input = layers.Input(shape=(2,), name="decoder_input")

In [61]:
x = layers.Dense(np.prod(shape_before_flattening))(decoder_input)
x = layers.Reshape(shape_before_flattening)(x)
x = layers.Conv2DTranspose(IMG_SIZE * 4, (3, 3), strides=2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(IMG_SIZE * 2, (3, 3), strides=2, activation='relu', padding='same')(x)
x = layers.Conv2DTranspose(IMG_SIZE, (3, 3), strides=2, activation='relu', padding='same')(x)

In [62]:
decoder_output = layers.Conv2D(1, (3, 3), strides=1, activation='sigmoid', padding='same', name="decoder_output")(x)

In [63]:
decoder = Model(decoder_input, decoder_output, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 dense_2 (Dense)             (None, 2048)              6144      
                                                                 
 reshape_2 (Reshape)         (None, 4, 4, 128)         0         
                                                                 
 conv2d_transpose_6 (Conv2DT  (None, 8, 8, 128)        147584    
 ranspose)                                                       
                                                                 
 conv2d_transpose_7 (Conv2DT  (None, 16, 16, 64)       73792     
 ranspose)                                                       
                                                                 
 conv2d_transpose_8 (Conv2DT  (None, 32, 32, 32)       1846

In [64]:
autoencoder = Model(encoder_input, decoder(encoder_output), name="autoencoder")
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 32)        320       
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 64)          18496     
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 4, 128)         73856     
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 encoder_output (Dense)      (None, 2)                 4098      
                                                                 
 decoder (Functional)        (None, 32, 32, 1)         

In [65]:
autoencoder.compile(optimizer="adam", loss="binary_crossentropy")

In [66]:
autoencoder.fit(
    x_train,
    x_train,
    epochs=3,
    batch_size=100,
    shuffle=True,
    validation_data=(x_test, x_test)
)

Epoch 1/3
600/600 [==============================] - 5s 8ms/step - loss: 0.3029 - val_loss: 0.2662
Epoch 2/3
600/600 [==============================] - 4s 7ms/step - loss: 0.2620 - val_loss: 0.2592
Epoch 3/3
600/600 [==============================] - 4s 7ms/step - loss: 0.2560 - val_loss: 0.2556
